In [2]:
!pip install nltk
import nltk
import re
%matplotlib inline
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
df0=pd.read_csv('scraped_data/crawling_0.csv')
df1=pd.read_csv('scraped_data/crawling_1.csv')
df2=pd.read_csv('scraped_data/crawling_2.csv')
df3=pd.read_csv('scraped_data/crawling_3.csv')
df4=pd.read_csv('scraped_data/crawling_4.csv')
df5=pd.read_csv('scraped_data/crawling_5.csv')
df6=pd.read_csv('scraped_data/crawling_6.csv')
df7=pd.read_csv('scraped_data/crawling_7.csv')
df8=pd.read_csv('scraped_data/crawling_8.csv')
df9=pd.read_csv('scraped_data/crawling_9.csv')
df10=pd.read_csv('scraped_data/crawling_10.csv')
df11=pd.read_csv('scraped_data/crawling_11.csv')
data_df=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

In [4]:
data_df.reset_index(drop=True, inplace=True)

In [5]:
selected_categories = data_df['class'].value_counts()[:20].index.tolist()
selected_categories

['Thời sự',
 'Thế giới',
 'Kinh doanh',
 'Thể thao',
 'Pháp luật',
 'Sức khỏe',
 'Cần biết',
 'Giáo dục',
 'Văn hóa',
 'Nhịp sống trẻ',
 'Giải trí',
 'Khoa học',
 'Du Lịch',
 'Bạn đọc làm báo',
 'Công nghệ',
 'Phóng sự',
 'Doanh nghiệp',
 'Nhà đất',
 'Giả - Thật',
 'Tuyển sinh']

In [6]:
data_df = data_df[data_df['class'].isin(selected_categories)]
data_df

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...,...
51995,/tham-sat-o-thai-nguyen-nghi-pham-thua-nhan-nh...,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
51996,/mobifone-dat-loi-nhuan-hon-6-000-ty-dong-tron...,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
51997,/chuan-bi-co-le-hoi-hoa-anh-dao-tai-tp-hcm-201...,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
51998,/xet-xu-vu-nu-sinh-giao-ga-bui-van-cong-lien-t...,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


In [7]:
data_df=data_df[~((data_df['title'].isnull())|
                  (data_df['description'].isnull())|
                  (data_df['content'].isnull()))]

In [8]:
data_df.drop(['links'],axis=1,inplace=True)

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
data_df.drop_duplicates(inplace=True)

<ipython-input-9-d6b7b2b2ae7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.drop_duplicates(inplace=True)


In [10]:
data_df

,title,description,content,class
0,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...
51995,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
51996,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
51997,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
51998,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


In [11]:
!pip install pyvi
!pip install gensim
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [12]:
data_df["content"][0]

'Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao quyền lực một cách hòa bình, êm thắm, ngày 8-1, ông Trump viết trên Twitter: "Trả lời câu hỏi của các bạn, tôi sẽ không tham dự lễ nhậm chức ngày 20-1".Dòng tweet này của ông liên tục biến động các con số về lượt người xem và chia sẻ cũng như bình luận. Nhiều người phản đối đã để lại bình luận không mấy thân thiện bên dưới như "không ai muốn ngài có mặt ở đó"...Theo Hãng tin Reuters, một nguồn tin am hiểu nội vụ cho biết ở Nhà Trắng đã có sự bàn bạc về việc ông Trump sẽ rời Washington ngày 19-1. Cũng theo nguồn tin này, ông Trump sẽ đến khu nghỉ dưỡng của mình ở bang Florida.Trong lịch sử Mỹ, có nhiều Tổng thống từng quyết định không dự lễ nhậm chức của người kế nhiệm.\xa0Tổng thống Richard Nixon rời Nhà Trắng khi Gerald Ford tuyên thệ. Ông Nixon từ chức, báo với Phó tổng thống của mình là Gerald Ford vào ngày 8-8 và ông Ford thay mình từ "ngày hôm nay".\xa0 Lúc 9h30 ngày 9-8, ông Nixon có bài phát biểu cuối cùng sau đó rời đi. Khi ông đáp

In [13]:
#' '.join(gensim.utils.simple_preprocess(data_df['content'][0]))
def gensim_utils_simple_preprocess(x):
    return ' '.join(gensim.utils.simple_preprocess(x))
data_df['content']=data_df['content'].apply(gensim_utils_simple_preprocess)
data_df['class']=data_df['class'].apply(gensim_utils_simple_preprocess)

<ipython-input-13-89f5c23e7282>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['content']=data_df['content'].apply(gensim_utils_simple_preprocess)
<ipython-input-13-89f5c23e7282>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['class']=data_df['class'].apply(gensim_utils_simple_preprocess)


In [14]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\\p{P}*)([p{L}.]*\\p{L}+)(\\p{P}*$)', r'\\1/\\2/\\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)



data_df['content']=data_df['content'].apply(chuan_hoa_dau_cau_tieng_viet)
data_df['class']=data_df['class'].apply(chuan_hoa_dau_cau_tieng_viet)

<ipython-input-14-05c41ae763d0>:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['content']=data_df['content'].apply(chuan_hoa_dau_cau_tieng_viet)
<ipython-input-14-05c41ae763d0>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['class']=data_df['class'].apply(chuan_hoa_dau_cau_tieng_viet)


In [15]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

data_df['content']=data_df['content'].apply(covert_unicode)
data_df['class']=data_df['class'].apply(covert_unicode)

<ipython-input-15-8cf2079d02a7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['content']=data_df['content'].apply(covert_unicode)
<ipython-input-15-8cf2079d02a7>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['class']=data_df['class'].apply(covert_unicode)


In [16]:
ViTokenizer.tokenize(data_df["content"][0])

'sau khi đưa ra cam_kết sẽ đảm_bảo chuyển_giao quyền_lực một_cách hòa bình êm thắm ngày ông trump viết trên twitter trả_lời câu hỏi của các bạn tôi sẽ không tham_dự lễ nhậm_chức ngày dòng tweet này của ông liên_tục biến_động các con_số về lượt người xem và chia_sẻ cũng như bình_luận nhiều người phản_đối đã để lại bình_luận không mấy thân_thiện bên dưới như không ai muốn ngài có_mặt đó theo hãng tin reuters một nguồn tin am_hiểu nội_vụ cho biết nhà trắng đã có sự bàn_bạc về việc ông trump sẽ rời washington ngày cũng theo nguồn tin này ông trump sẽ đến khu nghỉ_dưỡng của mình bang florida trong lịch_sử mỹ có nhiều tổng_thống từng quyết_định không dự lễ nhậm_chức của người kế_nhiệm tổng_thống richard nixon rời nhà trắng khi gerald ford tuyên_thệ ông nixon từ_chức báo với phó_tổng_thống của mình là gerald ford vào ngày và ông ford thay mình từ ngày hôm_nay lúc ngày ông nixon có bài phát_biểu cuối_cùng sau đó rời đi khi ông đáp máy_bay xuống california ông ford đã tuyên_thệ xong ngoài_ra cò

In [16]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)